In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
import time

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
X_train_bert = np.load('X_train_bert2.npy')
X_test_bert= np.load('X_test_bert2.npy')
y_train_rating= np.load('y_train_rating.npy')
y_test_rating = np.load('y_test_rating.npy')
y_train_sentiment= np.load('y_train_sentiment.npy')
y_test_sentiment = np.load('y_test_sentiment.npy')

In [19]:
print(X_train_bert.shape)

(2399668, 774)


## model

## entrainement

In [4]:
batch_size = 4096*4
#LE -1 est pour que les classes soient de 0 à 4
train_dataset_bert = TensorDataset(torch.tensor(X_train_bert),torch.tensor(y_train_rating-1, dtype=torch.long))
test_dataset_bert = TensorDataset(torch.tensor(X_test_bert),torch.tensor(y_test_rating-1, dtype=torch.long))
train_loader = DataLoader(train_dataset_bert, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset_bert, batch_size=batch_size, shuffle=False)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [37]:
nb_epoch = 15
#model = DEEPmodel(input_size=774, feature_size=256, nb_class=5, nb_couche=1).to(device)
model = DEEPmodel(input_size=774, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(nb_epoch):
    model.train()
    y_pred_all = []
    nb = 0
    total = 0
    
    for i, (x, y) in enumerate(train_loader):
        print(".",end="")
        x, y = x.to(device).float(), y.to(device)  # Move data to GPU
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred_all.append(y_pred.detach().cpu())  # Move to CPU for later operations
        loss = loss_fn(y_pred, y)
        nb += 1
        total += y.size(0)
        loss.backward()
        optimizer.step()
    print()
    print(f"epoch {epoch} :  loss : {loss.item():0.3f}")
    
    y_pred_all = torch.cat(y_pred_all)
    acc_train = accuracy_score(y_train_rating - 1, y_pred_all.argmax(1).numpy())
    
    model.eval()
    with torch.no_grad():
        y_pred = []
        for i, (x, y) in enumerate(test_loader):
            x = x.to(device).float()  # Move test data to GPU
            y_pred_batch = model(x).argmax(1)
            y_pred.append(y_pred_batch.cpu())  # Move to CPU for evaluation
        
        y_pred = torch.cat(y_pred)
        y_sentiment = y_pred.cpu().numpy().copy()
        y_sentiment[y_sentiment < 3] = 0
        y_sentiment[y_sentiment == 3] = 1
        y_sentiment[y_sentiment > 3] = 2
        acc_test_sent = accuracy_score(y_test_sentiment , y_sentiment)
        acc_test = accuracy_score(y_test_rating - 1, y_pred.numpy())
    
    print(f"epoch {epoch} :  train : {acc_train:0.3f}  test : {acc_test:0.3f} test sentiment {acc_test_sent:0.3f}")

....................

KeyboardInterrupt: 

In [7]:
train_dataset_bert_sent = TensorDataset(torch.tensor(X_train_bert),torch.tensor(y_train_sentiment, dtype=torch.long))
test_dataset_bert_sent = TensorDataset(torch.tensor(X_test_bert),torch.tensor(y_test_sentiment, dtype=torch.long))
train_loader_sent = DataLoader(train_dataset_bert_sent, batch_size=batch_size, shuffle=True)
test_loader_sent = DataLoader(test_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [8]:
nb_epoch = 25
#model = DEEPmodel(input_size=774, feature_size=256, nb_class=5, nb_couche=1).to(device)
model = DEEPmodel_2(input_size=774, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(nb_epoch):
    model.train()
    y_pred_all = []
    nb = 0
    total = 0
    
    for i, (x, y) in enumerate(train_loader_sent):
        print(".",end="")
        x, y = x.to(device).float(), y.to(device)  # Move data to GPU
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred_all.append(y_pred.detach().cpu())  # Move to CPU for later operations
        loss = loss_fn(y_pred, y)
        nb += 1
        total += y.size(0)
        loss.backward()
        optimizer.step()
    print()
    print(f"epoch {epoch} :  loss : {loss.item():0.3f}")
    
    y_pred_all = torch.cat(y_pred_all)
    acc_train = accuracy_score(y_train_sentiment , y_pred_all.argmax(1).numpy())
    
    model.eval()
    with torch.no_grad():
        y_pred = []
        for i, (x, y) in enumerate(test_loader_sent):
            x = x.to(device).float()  # Move test data to GPU
            y_pred_batch = model(x).argmax(1)
            y_pred.append(y_pred_batch.cpu())  # Move to CPU for evaluation
        
        y_pred = torch.cat(y_pred)
        acc_test = accuracy_score(y_test_sentiment, y_pred.numpy())
    
    print(f"epoch {epoch} :   test : {acc_test:0.3f}")

...................................................................................................................................................
epoch 0 :  loss : 0.419
epoch 0 :   test : 0.850
...................................................................................................................................................
epoch 1 :  loss : 0.397
epoch 1 :   test : 0.854
...................................................................................................................................................
epoch 2 :  loss : 0.379
epoch 2 :   test : 0.858
...................................................................................................................................................
epoch 3 :  loss : 0.394
epoch 3 :   test : 0.861
...................................................................................................................................................
epoch 4 :  loss : 0.385
epoch 4 :   test : 0.862
...............

In [9]:
torch.save(model.state_dict(), "Deep_bert.pth")

In [55]:
prediction[prediction < 2] = 0
prediction[prediction == 2] = 1
prediction[prediction > 2] = 2
accuracy_score(y_test_sentiment , prediction)

0.7905326903554992

In [56]:
accuracy_score(y_test_rating - 1, y_pred.numpy())

0.6824827434461768

In [57]:
y_pred

tensor([4, 4, 4,  ..., 4, 4, 4])

In [16]:
X_train_bert

array([[ 4.26827778e+00,  2.12690000e+04,  4.11363226e+00, ...,
        -3.08951586e-01,  1.19200364e-01, -8.06853548e-02],
       [ 4.63971121e+00,  3.72590000e+04,  4.11363226e+00, ...,
        -4.13514942e-01,  1.01978950e-01,  3.63112465e-02],
       [ 4.19334736e+00,  1.28370000e+04,  2.50000000e+00, ...,
        -4.71533537e-02,  1.23229310e-01,  1.23927303e-01],
       ...,
       [ 4.54850529e+00,  1.20090000e+04,  4.38628309e+00, ...,
        -6.88805133e-02,  4.48683769e-01,  9.02073644e-03],
       [ 2.50000000e+00,  0.00000000e+00,  2.50000000e+00, ...,
        -2.52590608e-02,  4.60583530e-02,  3.84012848e-01],
       [ 5.00000000e+00,  2.00000000e+00,  4.45370370e+00, ...,
        -1.48511499e-01,  3.81951034e-03,  6.31154597e-01]])